In [26]:
from paramiko import client
import pandas as pd
import folium

class ssh:
    client = None
 
    def __init__(self, address, username, password):
        print("Connecting to server.")
        self.client = client.SSHClient()
        self.client.set_missing_host_key_policy(client.AutoAddPolicy())
        self.client.connect(address, username=username, password=password, look_for_keys=False)

In [27]:
#zadaj prihlasovacie udaje na hpcdev02
connection = ssh("hpcdev02", "username", "passwd")

Connecting to server.


In [65]:
#Stanice AMS-2
stdin, stdout, stderr = connection.client.exec_command('mysql -u nwp -h mondo -e "select lat,lon, elev, name, mtime , vtime from si.si where si.si.ci=11;"')
result = stdout.read().decode('utf-8').strip("\n")
novy=result.splitlines()
st_frame=pd.DataFrame(columns=[splits for splits in novy[0].split("\t") if splits is not ""])
ll=1
for lines in novy[1:]:
    st_frame.loc[ll]=[splits for splits in lines.split("\t") if splits is not ""]
    ll+=1

In [66]:
#Stanice NMSKO
stdin, stdout, stderr = connection.client.exec_command('mysql -u nwp -h mondo -e "select lat,lon, elev, name, mtime , vtime from si.si where si.si.ci=78;"')
result = stdout.read().decode('utf-8').strip("\n")
novy=result.splitlines()
st_frame_NMSKO=pd.DataFrame(columns=[splits for splits in novy[0].split("\t") if splits is not ""])
ll=1
for lines in novy[1:]:
    st_frame_NMSKO.loc[ll]=[splits for splits in lines.split("\t") if splits is not ""]
    ll+=1

In [67]:
lat = st_frame.lat.astype(float)
lon = st_frame.lon.astype(float)
elev = st_frame.elev.astype(float)
name = st_frame.name

lat_NMSKO = st_frame_NMSKO.lat.astype(float)
lon_NMSKO = st_frame_NMSKO.lon.astype(float)
elev_NMSKO = st_frame_NMSKO.elev.astype(float)
name_NMSKO = st_frame_NMSKO.name

In [71]:
map = folium.Map(location=[48.85534, 19.7367], zoom_start=8, tiles="Stamen Terrain")


fgAMS = folium.FeatureGroup(name="AMS-2")

for lt, ln, el, name_st  in zip(lat, lon, elev, name):
    fgAMS.add_child(folium.CircleMarker(location=[lt, ln], radius = 6, popup=name_st+' '+str(el)+" m",
    fill_color='yellow', fill=True,  color = 'grey', fill_opacity=0.7))

fgNMSKO = folium.FeatureGroup(name="NMSKO")

for lt, ln, el, name_st  in zip(lat_NMSKO, lon_NMSKO, elev_NMSKO, name_NMSKO):
    fgNMSKO.add_child(folium.CircleMarker(location=[lt, ln], radius = 6, popup=name_st+' '+str(el)+" m",
    fill_color='blue', fill=True,  color = 'grey', fill_opacity=0.7))

map.add_child(fgAMS)
map.add_child(fgNMSKO)
map.add_child(folium.LayerControl())

map.save("MapaT.html")
display(map)